# Google Finance Scraping & Portfolio Simulation

---

**Author:** Kevin Milli

---

## Objective
The objective of this notebook is to develop a script for extracting valuable information from  [Google Finance](https://www.google.com/finance/).<br>
The extracted data will serve the purpose of simulating a portfolio, requiring market value, current value, stock name, and exchange information.

## Process Description
The process involves scraping data from Google Finance using Python libraries such as requests and BeautifulSoup.<br>
The data will then be utilized to simulate a portfolio, where the market value and current value of stocks, along with their names and exchanges, will be important factors.

Approach to the Problem
The approach adopted utilizes several Python libraries, including:
- `requests`
- `BeautifulSoup`
- `dataclasses`
- `tabulate`

Three main functions have been implemented:
1. `get_price_information`: Retrieve price information for a given stock ticker and exchange.
2. `get_fx_to_usd`: Retrieve the foreign exchange rate of a given currency to USD.
3. `display_portfolio_values`: Display the values of stocks in the given portfolio and the total portfolio value.

Three main classes have been implemented:
1. `Stock`: Represents a stock with attributes such as ticker symbol, exchange, price, currency, and USD price.
2. `Position`: Represents a position in a portfolio, consisting of a stock and its quantity.
3. `Portfolio`: Represents a portfolio containing a list of positions.

## Acknowledgments

Gratitude is expressed to the entire staff of [Google Finance](https://www.google.com/finance/) for their accessibility and the quality of the content provided. Their platform has facilitated the extraction of valuable data for portfolio simulation.

In [1]:
import requests as r
from bs4 import BeautifulSoup

from dataclasses import dataclass
from tabulate import tabulate

In [2]:
@dataclass
class Stock:
    """
    Represents a stock with attributes such as ticker symbol, exchange, price, currency, and USD price.

    Attributes:
        ticker (str): The ticker symbol of the stock.
        exchange (str): The stock exchange where the stock is listed.
        price (float): The current price of the stock.
        currency (str): The currency in which the price is denominated.
        usd_price (float): The price of the stock converted to USD, if necessary.
    """
    ticker: str
    exchange: str
    price: float = 0 
    currency: str = "USD"
    usd_price: float = 0

    def __post_info__(self):
        """
        Update price, currency, and USD price of the stock by fetching information from Google Finance.
        """
        price_info = get_price_information(self.ticker, self.exchange)
        self.price = price_info["price"]
        self.currency = price_info["currency"]
        self.usd_price = price_info["usd_price"]

    def __init__(self, ticker: str, exchange: str):
        """
        Initialize a Stock object with the provided ticker symbol and exchange.
        """
        self.ticker = ticker
        self.exchange = exchange
        self.__post_info__() 


In [3]:
@dataclass
class Position:
    """
    Represents a position in a portfolio, consisting of a stock and its quantity.

    Attributes:
        stock (Stock): The stock associated with the position.
        quantity (int): The quantity of the stock held in the position.
    """
    stock: Stock
    quantity: int
    

In [4]:
@dataclass
class Portfolio:
    """
    Represents a portfolio containing a list of positions.

    Attributes:
        positions (list[Position]): List of positions representing the stocks in the portfolio.
    """
    positions: list[Position]

    def get_total_value(self):
        """
        Calculate the total value of the portfolio.

        Returns:
            float: The total value of the portfolio.
        """
        total_value = 0

        for position in self.positions:
            total_value += position.quantity * position.stock.usd_price

        return total_value
    

In [5]:
def get_fx_to_usd(currency):
    """
    Retrieve the foreign exchange rate of a given currency to USD.

    Args:
        currency (str): The currency code for which to retrieve the exchange rate.

    Returns:
        float: The foreign exchange rate of the specified currency to USD.
    """
    fx_url = f"https://www.google.com/finance/quote/{currency}-USD"
    resp = r.get(fx_url)
    soup = BeautifulSoup(resp.content, "html.parser")

    fx_rate = soup.find("div", attrs={"data-last-price": True})
    fx = float(fx_rate["data-last-price"])
    return fx
    

In [6]:
def get_price_information(ticker, exchange):
    """
    Retrieve price information for a given stock ticker and exchange.

    Args:
        ticker (str): The ticker symbol of the stock.
        exchange (str): The stock exchange where the stock is listed.

    Returns:
        dict: A dictionary containing the following keys:
            - 'ticker' (str): The ticker symbol of the stock.
            - 'exchange' (str): The stock exchange where the stock is listed.
            - 'price' (float): The current price of the stock.
            - 'currency' (str): The currency in which the price is denominated.
            - 'usd_price' (float): The price of the stock converted to USD, if necessary.
    """
    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"
    resp = r.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    price_div = soup.find("div", attrs={"data-last-price": True})
    price = float(price_div["data-last-price"])
    currency = price_div["data-currency-code"]

    usd_price = price
    if currency != "USD":
        fx = get_fx_to_usd(currency)
        usd_price = round(price * fx, 2)

    return {
        "ticker": ticker,
        "exchange": exchange,
        "price": price,
        "currency": currency,
        "usd_price": usd_price
    }
    


In [7]:
def display_portfolio_values(portfolio):
    """
    Display the values of stocks in the given portfolio and the total portfolio value.

    Args:
        portfolio (Portfolio): An instance of the Portfolio class representing the portfolio.

    Raises:
        TypeError: If the provided object is not an instance of the Portfolio class.

    Prints:
        The ticker symbol, exchange, quantity, price, market value, and percentage allocation
        of each stock position in the portfolio, as well as the total portfolio value.
    """
    if not isinstance(portfolio, Portfolio):
        raise TypeError("Please provide an instance of type Portfolio")

    portfolio_value = portfolio.get_total_value()

    position_data = []

    for position in portfolio.positions:
        position_data.append([
            position.stock.ticker,
            position.stock.exchange,
            position.quantity,
            position.stock.usd_price,
            position.quantity * position.stock.usd_price,
            (position.quantity * position.stock.usd_price / portfolio_value) * 100
        ])

    print(tabulate(position_data,
                   headers=["Ticker", "Exchange", "Quantity", "Price", "Market Value", "% Allocation"],
                   tablefmt="psql", 
                   floatfmt=".2f"
                  ))

    print(f"Total Portfolio Value: {portfolio_value:,.2f}")

    

In [11]:
stock = Stock("SHOP", "TSE")
stock2 = Stock("AMZN", "NASDAQ")
stock3 = Stock("SHOP", "NYSE")
stock4 = Stock("MSFT", "NASDAQ")
stock5 = Stock("GOOGL", "NASDAQ")

In [13]:
portfolio = Portfolio([Position(stock, 10), Position(stock2, 5), Position(stock3, 20), Position(stock4, 12), Position(stock4, 7)])

In [14]:
display_portfolio_values(portfolio)

+----------+------------+------------+---------+----------------+----------------+
| Ticker   | Exchange   |   Quantity |   Price |   Market Value |   % Allocation |
|----------+------------+------------+---------+----------------+----------------|
| SHOP     | TSE        |         10 |   77.22 |         772.20 |           6.96 |
| AMZN     | NASDAQ     |          5 |  174.42 |         872.10 |           7.86 |
| SHOP     | NYSE       |         20 |   77.15 |        1543.00 |          13.90 |
| MSFT     | NASDAQ     |         12 |  416.42 |        4997.04 |          45.02 |
| MSFT     | NASDAQ     |          7 |  416.42 |        2914.94 |          26.26 |
+----------+------------+------------+---------+----------------+----------------+
Total Portfolio Value: 11,099.28
